In [2]:
import pandas as pd 
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import time
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_pickle('/content/drive/MyDrive/Grupo Facens DataScience/TCC/Grupo 1/Codes/Arquivos/INFLUD21-27-09-2021-TratadoWDummies.pkl')

In [4]:
y = df['SUPORT_VEN'] 
x = df.drop(['SUPORT_VEN'],axis=1)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [6]:
def model_run(x,y):
    model = {"Bernoulli" : BernoulliNB(), 'ComplementNB' : ComplementNB(), 'MultinomialNB':  MultinomialNB() } 
    
    acuracy = {}
    for M in model.keys():
    
        Model = model[M]
        Model.fit(X_train, y_train)
        y_pred = Model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        acuracy[M] = acc 
    
    return acuracy

def MultiConfusionMatrix(y_test, y_pred):
    labels = list(y_test.unique())
    cf = multilabel_confusion_matrix(y_test, y_pred, labels=labels) 
    matrix = {'Classe' : [], 
             'VP':[],
             'FP':[],
             'VN':[],
             'FN':[]}
    for x, Class in enumerate(labels): 
        VN = cf[x][0][0]
        FN = cf[x][1][0]
        VP = cf[x][1][1]
        FP = cf[x][0][1]
        matrix['Classe'].append(Class)
        matrix['VP'].append(VP)
        matrix['FP'].append(FP)
        matrix['VN'].append(VN)
        matrix['FN'].append(FN)
    return pd.DataFrame(matrix)

In [7]:
analise = {'Bernoulli': [],
            'ComplementNB': [],
            'MultinomialNB': []}

#Opção 1 - URL
acc = model_run(x, y)

analise['Bernoulli'].append(acc["Bernoulli"])
analise['ComplementNB'].append(acc['ComplementNB'])
analise['MultinomialNB'].append(acc['MultinomialNB'])

analise

{'Bernoulli': [0.5729283435371216],
 'ComplementNB': [0.5053138742778305],
 'MultinomialNB': [0.6151806501681469]}

In [ ]:
#Rodando Gridsearch p/ KNN
grid_params = {
    'n_neighbors' : [3,5,7],
    'weights' : ['uniform','distance'],
    'metric' : ['chebyshev','manhattan','euclidean'],
    'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']}

gs = GridSearchCV(KNeighborsClassifier(), grid_params, verbose = 1, cv = 2, n_jobs = 1)

gs_results = gs.fit(X_train, y_train)
gs_results.best_score_
gs_results.best_estimator_
gs_results.best_params_

Fitting 2 folds for each of 36 candidates, totalling 72 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
# Rodar Gridsearch com decisiontree
grid_params = {
    'criterion' : ['gini','entropy'],
    'splitter' :  ['best', 'random'],
    'max_depth' : [3,4,7,8,9],
    'max_features' : ['auto', 'sqrt', 'log2']}

gs = GridSearchCV(DecisionTreeClassifier(), grid_params, verbose = 1, cv = 3, n_jobs = -1)

gs_results = gs.fit(X_train, y_train)
gs_results.best_score_
gs_results.best_estimator_
gs_results.best_params_

In [ ]:
# Rodar Gridsearch com Logistic Regression Classifier
grid_params = { 
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'multi_class': ['auto', 'ovr', 'multinomial']
}

gs = GridSearchCV(LogisticRegression(), grid_params, verbose = 1, cv = 3, n_jobs = -1)

gs_results = gs.fit(X_train, y_train)
gs_results.best_score_
gs_results.best_estimator_
gs_results.best_params_

In [9]:
model = MultinomialNB()
scores = cross_val_score(model, x, y, cv=5)
print('Acurácia de MultinomialNB:', scores.mean())
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(MultiConfusionMatrix(y_test, y_pred))
print('-'*80)
print('\n')

model = KNeighborsClassifier(algorithm='auto', metric='manhattan', n_neighbors= 5, weights='distance')
scores = cross_val_score(model, x, y, cv=5)
print('Acurácia de Knn:', scores.mean())
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(MultiConfusionMatrix(y_test, y_pred))
print('-'*80)
print('\n')

model = DecisionTreeClassifier(criterion='entropy', max_depth = 9, max_features='auto', splitter='random')
scores = cross_val_score(model, x, y, cv=5)
print('Acurácia de Decision Tree:', scores.mean())
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(MultiConfusionMatrix(y_test, y_pred))
print('-'*80)
print('\n')

model = RandomForestClassifier(max_features='log2', n_estimators= 700)
scores = cross_val_score(model, x, y, cv=5)
print('Acurácia de Random Forest:', scores.mean())
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(MultiConfusionMatrix(y_test, y_pred))
print('-'*80)
print('\n')
    
model = LogisticRegression(multi_class='auto', solver='liblinear')
scores = cross_val_score(model, x, y, cv=5)
print('Acurácia de Logistic Regression:', round(scores.mean(),4),' - ', round(scores.std(),2))
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(MultiConfusionMatrix(y_test, y_pred))
print('-'*80)
print('\n')


Acurácia de MultinomialNB: 0.6136573074485354
   Classe     VP     FP     VN     FN
0       1   1257   1914  69689  19916
1       2  49851  26734   9070   7121
2       3   5966   7054  71091   8665
--------------------------------------------------------------------------------


Acurácia de Knn: 0.553213894587874
   Classe     VP     FP     VN     FN
0       1   5489  13166  58437  15684
1       2  42166  23507  12297  14806
2       3   3606   4842  73303  11025
--------------------------------------------------------------------------------


Acurácia de Decision Tree: 0.621834758695367
   Classe     VP     FP     VN     FN
0       1    171    263  71340  21002
1       2  54145  31335   4469   2827
2       3   3514   3348  74797  11117
--------------------------------------------------------------------------------


Acurácia de Random Forest: 0.5965981259623317
   Classe     VP     FP     VN     FN
0       1   3094   5938  65665  18079
1       2  48016  26624   9180   8956
2       3